In [1]:
import os
import PIL 
import PIL.Image
import numpy as np
#!pip install -U insightface
#!pip install onnxruntime
from insightface.app import FaceAnalysis

In [ ]:
#download from
#!wget https://www.dropbox.com/scl/fi/b9y03enpi01zfb36ckyir/personas.zip?rlkey=19wypqmaps5b5n28bz3gi7xg7&st=r52y68rh&dl=0 -O persona.zip

In [4]:
datapath = '/home/data/personas'
ffaces = os.path.join(datapath, 'faces.txt')

# Initialize face analysis model
app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])  # Use 'CUDAExecutionProvider' for GPU
app.prepare(ctx_id=0)  # ctx_id=-1 for CPU, 0 for GPU

def get_face_embedding(image_path):
    """Extract face embedding from an image"""
    img = PIL.Image.open(image_path)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")    
    faces = app.get(np.array(img))    
    if len(faces) < 1:
        raise ValueError("No faces detected in the image")
    if len(faces) > 1:
        print("Warning: Multiple faces detected. Using first detected face")
    
    return faces[0].embedding

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/jsaavedr/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/jsaavedr/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/jsaavedr/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/jsaavedr/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/jsaavedr/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

In [5]:
embs = []
names = []
compute = True 
with open(ffaces)  as f: 
    for line in f : 
        iname, icl = line.split()
        iname = os.path.join(datapath, iname.strip())
        print(iname)
        names.append(icl)
        if compute :
            emb = get_face_embedding(iname)
            if len(embs) == 0 :
                embs = emb
            else :
                embs = np.vstack([embs, emb])

print(embs.shape)
np.save("face_embs.npy", embs)
# embs = np.load("face_embs.npy") 
#similarity search
embs_norm = embs / np.linalg.norm(embs, ord = 2, axis = 1,  keepdims = True)
sim = embs_norm @ np.transpose(embs_norm)
idx_sort =  np.argsort(-sim, axis = 1)
print(idx_sort)
for  idx, row in enumerate(idx_sort) :    
    result = [(names[i], sim[idx, i]) for i in row[:5] ]
    print(result)


/home/data/personas/paulina/paulina2.png
/home/data/personas/paulina/paulina1.jpeg
/home/data/personas/dicaprio/dicaprio_2.png
/home/data/personas/dicaprio/dicaprio.jpg
/home/data/personas/barack/barack.jpeg
/home/data/personas/barack/barack_2.jpeg
/home/data/personas/barack/face_2.jpeg
/home/data/personas/pancho/pancho.jpeg
/home/data/personas/pancho/pancho2.jpeg
/home/data/personas/thalia/thalia1.jpg
/home/data/personas/thalia/thalia2.jpg
(11, 512)
[[ 0  1  2  3 10  7  8  9  5  4  6]
 [ 1  0  2  7  3  8  9 10  6  5  4]
 [ 2  3  1  0  6  4  5  7  8  9 10]
 [ 3  2  6  4  0  1  5  7  8 10  9]
 [ 4  6  5  3  2  7  9 10  8  0  1]
 [ 5  6  4  2  3  9 10  7  8  0  1]
 [ 6  5  4  3  2  9 10  7  1  0  8]
 [ 7  8  1  0  4  2  3  9  6  5 10]
 [ 8  7  1  0  4  9 10  5  2  3  6]
 [ 9 10  4  6  5  1  8  7  0  2  3]
 [10  9  0  4  6  5  1  8  7  2  3]]
[('paulina', np.float32(1.0000001)), ('paulina', np.float32(0.6209116)), ('dicaprio', np.float32(0.14419979)), ('dicaprio', np.float32(0.11881716)),